In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv(r"C:\Users\Dell\Downloads\merged_timeseries_per_reading.csv", encoding='utf-8-sig')
print(f"Kích thước dữ liệu: {df.shape}")
print(f"Các cột: {df.columns.tolist()}")



Kích thước dữ liệu: (8591130, 21)
Các cột: ['timestamp', 'temp', 'humid', 'co2', 'light', 'class', 'trip_id', 'fruit_cate', 'latitude', 'longitude', 'Route', 'temperature_C', 'humidity_%', 'dew_point_C', 'pressure_hPa', 'wind_speed_kmh', 'precipitation_mm', 'trip_id_raw', 'expected_delay_min', 'weight', 'unit_quantity']


# Thông tin cơ bản về data

In [16]:
print("=== THÔNG TIN CƠ BẢN VỀ DỮ LIỆU ===")
print(f"Kích thước: {df.shape}")
print(f"Thời gian: từ {df['timestamp'].min()} đến {df['timestamp'].max()}")

# Kiểm tra kiểu dữ liệu và giá trị thiếu
print("\n=== KIỂU DỮ LIỆU VÀ GIÁ TRỊ THIẾU ===")
df_info = pd.DataFrame({
    'kiểu_dữ_liệu': df.dtypes,
    'số_lượng_giá_trị_thiếu': df.isnull().sum(),
    'tỷ_lệ_giá_trị_thiếu': (df.isnull().sum() / len(df)) * 100,
    'số_lượng_giá_trị_duy_nhất': df.nunique()
})
print(df_info)

# Thông tin về các chuyến hàng
print("\n=== THÔNG TIN CHUYẾN HÀNG ===")
print(f"Số chuyến duy nhất: {df['trip_id'].unique()}")
print(f"Loại trái cây: {df['fruit_cate'].unique()}")
print(f"Tuyến đường: {df['Route'].unique()}")

=== THÔNG TIN CƠ BẢN VỀ DỮ LIỆU ===
Kích thước: (8591130, 22)
Thời gian: từ 2024-03-14 02:45:41 đến 2024-06-01 01:47:00

=== KIỂU DỮ LIỆU VÀ GIÁ TRỊ THIẾU ===
                      kiểu_dữ_liệu  số_lượng_giá_trị_thiếu  \
timestamp           datetime64[ns]                       0   
temp                       float64                       0   
humid                      float64                       0   
co2                        float64                       0   
light                      float64                       0   
class                       object                       0   
trip_id                     object                       0   
fruit_cate                  object                       0   
latitude                   float64                       0   
longitude                  float64                       0   
Route                       object                       0   
temperature_C              float64                       0   
humidity_%                 float64 

# Chuyển đổi kiểu dữ liệu và sắp xếp

In [17]:
# Chuyển đổi cột timestamp sang kiểu datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sắp xếp dữ liệu theo timestamp và trip_id để đảm bảo thứ tự thời gian
df = df.sort_values(['trip_id', 'timestamp']).reset_index(drop=True)

# Kiểm tra timestamp trùng lặp trong cùng chuyến
df['is_timestamp_dup'] = df.groupby('trip_id')['timestamp'].transform(lambda x: x.duplicated())
timestamp_trùng_lặp = df['is_timestamp_dup'].sum()
print(f"Số timestamp trùng lặp trong chuyến: {timestamp_trùng_lặp}")

# Kiểm tra các cột có giá trị không đổi
các_cột_không_đổi = [cột for cột in df.columns if df[cột].nunique() == 1]
print(f"Các cột không đổi: {các_cột_không_đổi}")

# Hiển thị mẫu dữ liệu đã xử lý
print("\n=== MẪU DỮ LIỆU ĐÃ XỬ LÝ ===")
print(df[['timestamp', 'trip_id', 'fruit_cate', 'temp', 'humid', 'co2']].head())


Số timestamp trùng lặp trong chuyến: 0
Các cột không đổi: ['is_timestamp_dup']

=== MẪU DỮ LIỆU ĐÃ XỬ LÝ ===
            timestamp     trip_id fruit_cate       temp      humid         co2
0 2024-03-28 06:04:53  TRIP_00126     Tomato  22.888812  90.159451  320.601052
1 2024-03-28 06:14:53  TRIP_00126     Tomato  22.985532  89.737415  321.008643
2 2024-03-28 06:24:53  TRIP_00126     Tomato  22.825901  90.475185  321.356256
3 2024-03-28 06:34:53  TRIP_00126     Tomato  23.087722  89.628797  321.642294
4 2024-03-28 06:44:53  TRIP_00126     Tomato  23.110701  90.298860  322.181717


# Xử lý giá trị null


In [18]:
# Đếm số lượng null ở mỗi cột
null_counts = df.isnull().sum()

print("Số lượng dòng bị thiếu dữ liệu (Null/NaN):")
# Chỉ hiển thị các cột có chứa null để dễ nhìn
print(null_counts[null_counts > 0]) 

# Nếu in ra Series rỗng (Empty Series) nghĩa là dữ liệu đã sạch, không có null
if null_counts.sum() == 0:
    print("Tuyệt vời! File này không có giá trị null nào.")

Số lượng dòng bị thiếu dữ liệu (Null/NaN):
Series([], dtype: int64)
Tuyệt vời! File này không có giá trị null nào.


In [19]:
# Lưu lại kích thước trước khi xóa để so sánh
rows_before = df.shape[0]

# Xóa các dòng có ít nhất 1 giá trị null
df_cleaned = df.dropna()

rows_after = df_cleaned.shape[0]
rows_dropped = rows_before - rows_after

print(f"Số dòng trước khi xử lý: {rows_before}")
print(f"Số dòng sau khi drop null: {rows_after}")
print(f"Đã xóa bỏ: {rows_dropped} dòng chứa null.")

Số dòng trước khi xử lý: 8591130
Số dòng sau khi drop null: 8591130
Đã xóa bỏ: 0 dòng chứa null.


# Xuất file kết quả

In [20]:
output_path = r"C:\Users\Dell\Downloads\merged_timeseries_cleaned.csv"

df_cleaned.to_csv(output_path, index=False)

print(f"Đã lưu file sạch tại: {output_path}")

Đã lưu file sạch tại: C:\Users\Dell\Downloads\merged_timeseries_cleaned.csv
